In [6]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os

In [7]:


def read_input( train_path,test_path ):
    train = pd.read_csv(train_path)
    #print(train.head())
    print('df.shape :',train.shape)
    test = pd.read_csv(test_path)
    #print(test.head())
    print('df.shape :',test.shape)
    return train,test

def spilt_data( train,test ):
    value_columns = []
    for i in range(1,50):
        string = 'value_'+str(i)
        value_columns.append(string)   

    train_y = train[value_columns]
    test_y = test[value_columns]

    for col in value_columns:
        train.pop(col)
        test.pop(col)

    train_x = train
    test_x = test
    
    return train_y,test_y,train_x,test_x

In [8]:
def output(outputfile,predlist):
    np.savetxt(outputfile, predlist, delimiter=",")
    
def getscore(test_y,predict_test):
    #compute Accuracy
    tmp = abs( test_y - predict_test )/abs( test_y )
    #print(tmp)
    #print(type(tmp))
    #print("-----------------------------------------------------------")
    tmp[tmp >1] =1
    #print(tmp**2)
    square = tmp**2
    #print(square.shape[1])
    squ_sum = square.sum()
    #print(sum1)
    n = square.shape[0] * square.shape[1]
    #print(n)
    score = 100 - 100 * (np.sqrt(squ_sum/n))
    print("accuracy:" + str(score))
    
    return score,squ_sum

In [34]:
from sklearn.metrics import confusion_matrix
def customedscore(preds, dtrain):
    label = dtrain.get_label()
    tmp = abs( label - preds )/abs( label )
    tmp[tmp >1] =1
    square = tmp**2
    squ_sum = square.sum()
    
    return 'FSCORE',float(squ_sum), True

In [38]:
import lightgbm as lgb

def lgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name):
    lgb_len = len(test_y.T)
    num_leaves = 31
    num_trees = 100
    objective = 'regression'
    metric = 'rmse'
    param = {'num_leaves':num_leaves, 'num_trees':num_trees, 'objective':objective}
    param['metric'] = metric
    num_round = 10
    predict_test = []
    for i in range(lgb_len):
        
        train_data = lgb.Dataset(data=train_x,label=train_y.T[i])
        test_data = lgb.Dataset(data=test_x,label=test_y.T[i])
        #bst = lgb.train(param, train_data, num_round, valid_sets=[test_data],feval=customedscore,early_stopping_rounds=10)
        bst = lgb.train(param, train_data, num_round, valid_sets=[test_data],feval=customedscore)
        #ypred = bst.predict(test_x, num_iteration=bst.best_iteration)
        
        predict_test.append(bst.predict(test_x,num_iteration=bst.best_iteration))

    predict_test = np.array(predict_test).T
    #print(predict_test)
    testoutput = testout_path + "\\"+ test_file_name +"_lbg" +"_nl_" + str(num_leaves)  + "nm_" + str(num_trees) +'.csv' 
    errorout = testout_path + "\\"+ test_file_name +"_lbg" +"_nl_" + str(num_leaves)  + "nm_" + str(num_trees) +'_error.csv' 
    output(testoutput,(abs(predict_test)))
    output(errorout,(abs(predict_test - test_y)/abs( test_y ))**2)

    score,squ_sum = getscore(test_y,predict_test)
    """
    if (score > max_score):
        max_score = score
        best_md = md
        best_l = l
        best_n = n
    """

    #print("max_score:" +str(max_score))
    #print("best_md:" +str(best_md))
    #print("best_l:" +str(best_l))
    #print("best_n:" +str(best_n))
    return squ_sum
    

In [39]:

train_paths=[]
test_paths=[]
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv')
#train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv')
#train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv')
#train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv')
#train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_tt1p0v25c.csv')

#train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99v125c_base_400.csv')
# train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99vm40c_base_400.csv')
# train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81v125c_base_400.csv')
# train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81vm40c_base_400.csv')
# train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_tt0p9v25c_base_400.csv')

test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9vm40c_beta_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_tt1p0v25c_beta_100.csv')

# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99v125c_alpha_100.csv')
# test_paths.append( r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99vm40c_alpha_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81v125c_alpha_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81vm40c_alpha_100.csv')
# test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_tt0p9v25c_alpha_100.csv')

for i in range(len(test_paths)):
    print(train_paths[i])
    print(test_paths[i])




C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv


In [40]:
squ_sum = 0.0
n = 0
for i in range(len(test_paths)):
    print( "train_file:" + train_paths[i] )
    print( "test_file:" + test_paths[i] )
    train,test = read_input( train_paths[i],test_paths[i] )
    train_y,test_y,train_x,test_x = spilt_data( train,test )
    
    n = n + test_y.shape[0] * test_y.shape[1]
    print("n:" + str(n))
    #dataframe -> numpy.array.value
    train_x = train_x.values
    train_y = train_y.values
    test_x = test_x.values
    test_y = test_y.values
    testout_path = r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test_record'
    file,filesplit = os.path.split(test_paths[i])
    test_file_name = filesplit.split('.')[0]
    #print(test_file_name)
    squ_sum = squ_sum+lgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name)


score = 100 - 100 * (np.sqrt(squ_sum/n))
print("total_score:" + str(score))
    
    
    

train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
df.shape : (18858, 74)
df.shape : (5658, 74)
n:277242
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2271
[LightGBM] [Info] Number of data points in the train set: 18858, number of used features: 15
[LightGBM] [Info] Start training from score 0.037923
[1]	valid_0's rmse: 0.0358044	valid_0's FSCORE: 2693.48
[2]	valid_0's rmse: 0.0341292	valid_0's FSCORE: 2646.26
[3]	valid_0's rmse: 0.0326189	valid_0's FSCORE: 2605.54
[4]	valid_0's rmse: 0.0308169	valid_0's FSCORE: 2560.62
[5]	valid_0's rmse: 0.0297186	valid_0's FSCORE: 2542.67
[6]	valid_0's rmse: 0.0290759	v

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.



[41]	valid_0's rmse: 0.0274405	valid_0's FSCORE: 1500.29
[42]	valid_0's rmse: 0.0274244	valid_0's FSCORE: 1498.03
[43]	valid_0's rmse: 0.0274273	valid_0's FSCORE: 1487.21
[44]	valid_0's rmse: 0.0274493	valid_0's FSCORE: 1485
[45]	valid_0's rmse: 0.0274559	valid_0's FSCORE: 1481.09
[46]	valid_0's rmse: 0.0274892	valid_0's FSCORE: 1483.96
[47]	valid_0's rmse: 0.027491	valid_0's FSCORE: 1481.6
[48]	valid_0's rmse: 0.0274872	valid_0's FSCORE: 1481.45
[49]	valid_0's rmse: 0.0275389	valid_0's FSCORE: 1476.25
[50]	valid_0's rmse: 0.0275225	valid_0's FSCORE: 1474.67
[51]	valid_0's rmse: 0.0275357	valid_0's FSCORE: 1470.54
[52]	valid_0's rmse: 0.0275583	valid_0's FSCORE: 1470.05
[53]	valid_0's rmse: 0.0275316	valid_0's FSCORE: 1472.86
[54]	valid_0's rmse: 0.0275507	valid_0's FSCORE: 1473.81
[55]	valid_0's rmse: 0.0275595	valid_0's FSCORE: 1470.4
[56]	valid_0's rmse: 0.0275807	valid_0's FSCORE: 1474.58
[57]	valid_0's rmse: 0.0275749	valid_0's FSCORE: 1474.18
[58]	valid_0's rmse: 0.0276323	valid

[LightGBM] [Info] Start training from score 0.042753
[1]	valid_0's rmse: 0.0395469	valid_0's FSCORE: 2541.03
[2]	valid_0's rmse: 0.037751	valid_0's FSCORE: 2520.27
[3]	valid_0's rmse: 0.0363449	valid_0's FSCORE: 2510.92
[4]	valid_0's rmse: 0.0346172	valid_0's FSCORE: 2489.45
[5]	valid_0's rmse: 0.0335842	valid_0's FSCORE: 2485.26
[6]	valid_0's rmse: 0.0323234	valid_0's FSCORE: 2472.68
[7]	valid_0's rmse: 0.0316247	valid_0's FSCORE: 2470.5
[8]	valid_0's rmse: 0.0311147	valid_0's FSCORE: 2467.23
[9]	valid_0's rmse: 0.0306794	valid_0's FSCORE: 2465.2
[10]	valid_0's rmse: 0.0303617	valid_0's FSCORE: 2460.98
[11]	valid_0's rmse: 0.0301759	valid_0's FSCORE: 2454.73
[12]	valid_0's rmse: 0.0299843	valid_0's FSCORE: 2447.26
[13]	valid_0's rmse: 0.0298902	valid_0's FSCORE: 2437.03
[14]	valid_0's rmse: 0.0296503	valid_0's FSCORE: 2412.34
[15]	valid_0's rmse: 0.0296522	valid_0's FSCORE: 2390.37
[16]	valid_0's rmse: 0.0296009	valid_0's FSCORE: 2355.76
[17]	valid_0's rmse: 0.0296497	valid_0's FSCORE

[51]	valid_0's rmse: 0.0293343	valid_0's FSCORE: 1341.08
[52]	valid_0's rmse: 0.0293469	valid_0's FSCORE: 1336.67
[53]	valid_0's rmse: 0.0294499	valid_0's FSCORE: 1332.41
[54]	valid_0's rmse: 0.0294553	valid_0's FSCORE: 1323.04
[55]	valid_0's rmse: 0.029457	valid_0's FSCORE: 1321.59
[56]	valid_0's rmse: 0.0294386	valid_0's FSCORE: 1310.62
[57]	valid_0's rmse: 0.0294425	valid_0's FSCORE: 1302.43
[58]	valid_0's rmse: 0.029442	valid_0's FSCORE: 1301.83
[59]	valid_0's rmse: 0.0294332	valid_0's FSCORE: 1304.8
[60]	valid_0's rmse: 0.0294317	valid_0's FSCORE: 1304.52
[61]	valid_0's rmse: 0.0294293	valid_0's FSCORE: 1305
[62]	valid_0's rmse: 0.0294246	valid_0's FSCORE: 1304.9
[63]	valid_0's rmse: 0.0294219	valid_0's FSCORE: 1305.17
[64]	valid_0's rmse: 0.0294843	valid_0's FSCORE: 1291.46
[65]	valid_0's rmse: 0.0295216	valid_0's FSCORE: 1291.11
[66]	valid_0's rmse: 0.0295224	valid_0's FSCORE: 1289.97
[67]	valid_0's rmse: 0.0295501	valid_0's FSCORE: 1289.73
[68]	valid_0's rmse: 0.0296082	valid_0

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2271
[LightGBM] [Info] Number of data points in the train set: 18858, number of used features: 15
[LightGBM] [Info] Start training from score 0.113919
[1]	valid_0's rmse: 0.0885129	valid_0's FSCORE: 2447.99
[2]	valid_0's rmse: 0.0822063	valid_0's FSCORE: 2413.97
[3]	valid_0's rmse: 0.0763525	valid_0's FSCORE: 2368.51
[4]	valid_0's rmse: 0.0714549	valid_0's FSCORE: 2342.6
[5]	valid_0's rmse: 0.0673917	valid_0's FSCORE: 2331.2
[6]	valid_0's rmse: 0.0635589	valid_0's FSCORE: 2307.43
[7]	valid_0's rmse: 0.0604367	valid_0's FSCORE: 2293.99
[8]	valid_0's rmse: 0.0577253	valid_0's FSCORE: 2281.51
[9]	valid_0's rmse: 0.055751	valid_0's FSCORE: 2277.19
[10]	valid_0's rmse: 0.0537583	valid_0's FSCORE: 2269.02
[11]	valid_0's rmse: 0.0523454	va

[44]	valid_0's rmse: 0.0455019	valid_0's FSCORE: 1107.63
[45]	valid_0's rmse: 0.0455936	valid_0's FSCORE: 1068.88
[46]	valid_0's rmse: 0.0455744	valid_0's FSCORE: 1039.67
[47]	valid_0's rmse: 0.0455852	valid_0's FSCORE: 1033.52
[48]	valid_0's rmse: 0.045594	valid_0's FSCORE: 1008.39
[49]	valid_0's rmse: 0.045682	valid_0's FSCORE: 993.253
[50]	valid_0's rmse: 0.0456525	valid_0's FSCORE: 985.287
[51]	valid_0's rmse: 0.0456879	valid_0's FSCORE: 980.144
[52]	valid_0's rmse: 0.0456854	valid_0's FSCORE: 957.284
[53]	valid_0's rmse: 0.0457493	valid_0's FSCORE: 945.749
[54]	valid_0's rmse: 0.0457753	valid_0's FSCORE: 934.405
[55]	valid_0's rmse: 0.0458162	valid_0's FSCORE: 915.509
[56]	valid_0's rmse: 0.0458749	valid_0's FSCORE: 903.142
[57]	valid_0's rmse: 0.045887	valid_0's FSCORE: 915.219
[58]	valid_0's rmse: 0.0458171	valid_0's FSCORE: 909.891
[59]	valid_0's rmse: 0.0458048	valid_0's FSCORE: 901.26
[60]	valid_0's rmse: 0.0458112	valid_0's FSCORE: 900.193
[61]	valid_0's rmse: 0.0458197	vali

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide


accuracy:55.45239606430102
total_score:55.45239606430102


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
